## Nutrient Demands



### Introduction



In our last project we used data to estimate systems of food demand
using different datasets.  An output from that project was as set of
`cfe.Result` objects; these bundle together both data and the results
from the demand system estimation, and can be used for prediction as
well.  

Here we&rsquo;ll explore some of the uses of the `cfe.Result` class, using
an instance created using data from the Indian ICRISAT data.

If you don&rsquo;t already have the latest version of the `cfe` package
installed, grab it, along with some dependencies:



In [1]:
!pip install CFEDemands --upgrade
#!pip install xarray
#!pip install dvc
#!pip install h5netcdf

In [1]:
import pandas as pd
import cfe

We&rsquo;ll get data from two places.  First, basic data, including a food
 conversion table and recommended daily intakes table can be found in
 a google spreadsheet.



In [1]:
DataURL = 'https://docs.google.com/spreadsheets/d/13Ig5hZif-NSHtgkKRp_cEgKXk0lOsdUB2BAD6O_FnRo'
key = DataURL.split('/')[-1]

You&rsquo;ll also need to be sure that your credentials are decrypted:



In [1]:
!gpg --batch --passphrase "SECRET PASSPHRASE" -d ../students.json.gpg > ../students.json

An instance `r` of `cfe.Result` can be made persistent with
 `r.to_dataset('my_result.ds')`, which saves the instance &ldquo;on disk&rdquo; in
 [NetCDF format](https://en.wikipedia.org/wiki/NetCDF), and can be loaded using `cfe.from_dataset`.  We use
 this method below to load data and demand system estimated for the
 Indian ICRISAT villages:



In [1]:
r = cfe.from_dataset('./Indian ICRISAT_small.ds')

r

Pick prices that were prevailing at some time and place; e.g.,



In [1]:
# Year 1975, village A
t,m = 75,1

p = r.prices.sel(t=t,m=m,drop=True)
phat = p.to_dataframe('i').dropna()
phat

So, we have demands for median household at that time and place



In [1]:
xbar = r.get_predicted_expenditures().sum('i')
median_x = xbar.where(xbar>0).sel(t=t,m=m).median('j') # Budget (median household)

xhat = r.get_predicted_expenditures().sel(t=t,m=m,drop=True).mean('j')

xhatdf = xhat.to_dataframe('i').dropna()

qhat = xhatdf.div(phat,axis=1).dropna()

qhat

Finally, define a function to change a single price in the vector $p$:



In [1]:
def my_prices(p0,p=p,i='Coconut'):
    p = p.copy()
    p.loc[i] = p0
    return p

### Mapping to Nutrients



We&rsquo;ve seen how to map prices and budgets into vectors of consumption
 quantities using `cfe.Result.demands`.  Next we want to think about
 how to map these into bundles of *nutrients*.  The information needed
 for the mapping comes from a &ldquo;Food Conversion Table&rdquo; (or database,
 such as the [USDA Nutrition DataBase](https://ndb.nal.usda.gov/ndb/doc/apilist/API-FOOD-REPORTV2.md)).   Here we&rsquo;ve adapted (with
 thanks to Alessandro Tarozzi of the Universitat Pompeu Fabra) a food
 conversion table developed by Gopalan et al (1980) to the foods
 reported in the Indian ICRISAT data:



In [1]:
import eep153_tools

fct = eep153_tools.read_sheets(key,
                               json_creds='../students.json',
                               sheet='FCT').set_index('i')

We need the index of the Food Conversion Table (FCT) to match up with
 the index of the vector of quantities demanded.   To manage this we
 make use of the `align` method for `pd.DataFrames`:



In [1]:
# Create a new FCT and vector of consumption that only share rows in common:
fct0,c0 = fct.align(qhat,axis=0,join='inner')
print(fct0.index)

Now, since rows of `fct0` and `c0` match, we can obtain nutritional
 outcomes from the inner (or dot, or matrix) product of the transposed
 `fct0` and `c0`:



In [1]:
# The @ operator means matrix multiply
N = fct0.T@c0

# NB: ICRISAT data is *Annual* consumption
N

Of course, since we can compute the nutritional content of a vector of
 consumption goods `c0`, we can also use our demand functions to
 compute nutrition as a *function* of prices and budget.



In [1]:
def nutrient_demand(x,p):
    c = r.demands(x,p)
    fct0,c0 = fct.align(c,axis=0,join='inner')
    N = fct0.T@c0

    N = N.loc[~N.index.duplicated()]
    
    return N

With this `nutrient_demand` function in hand, we can see how nutrient
 outcomes vary with budget, given prices:



In [1]:
import numpy as np
import matplotlib.pyplot as plt

### If you're getting lots of not useful warnings you want to ignore
# import warnings
# warnings.filterwarnings('ignore',category=UserWarning)
####################################################################

X = np.linspace(median_x/5,median_x*5,50)

UseNutrients = ['Protein','Calories','Iron','Calcium']

plt.plot(X,pd.concat({myx:np.log(nutrient_demand(myx,p))[UseNutrients] for myx in X},axis=1).T)
plt.legend(UseNutrients)
plt.xlabel('log budget')
plt.ylabel('log nutrient')

Now how does nutrition vary with prices?



In [1]:
USE_GOOD = 'Jowar/Sorghum'

ref_price = r.prices.sel(i=USE_GOOD,t=t,m=m,drop=True)

P = np.linspace(ref_price/10,ref_price*10,50)

plt.plot(P,pd.DataFrame({p0:np.log(nutrient_demand(median_x,my_prices(p0,i=USE_GOOD)))[UseNutrients] for p0 in P}).T)
plt.legend(UseNutrients)
plt.xlabel('log price')
plt.ylabel('log nutrient')

## Nutritional requirements



Individuals have nutritional requirements established by nutrition
scientists.  Here are some standards established by the Indian
National Institute of Nutrition:



### Nutritional needs of individuals



#### Recommended Daily Allowances: India



| Sex-Age|Calories|Protein|Fat|Calcium|Iron|Betacarotene|Thiamine|Riboflavin|Niacin|Ascorbic Acid|
|---|---|---|---|---|---|---|---|---|---|---|
| C 0-0.5||||500|||.2|.3||25|
| C 0.5-1|||19|500|5|2800|.3|.4||25|
| C 1-3|1060|16.7|27|600|9|3200|.5|.6|8|40|
| C 4-6|1350|20.1|25|600|13|3200|.7|.8|11|40|
| C 7-9|1690|29.5|30|600|16|4800|.8|1.|13|40|
| B 10-12|2190|39.9|35|800|21|4800|1.1|1.3|15|40|
| G 10-12|2010|40.4|35|800|27|4800|1.|1.2|13|40|
| B 13-15|2750|54.3|45|800|32|4800|1.4|1.6|16|40|
| G 13-15|2330|51.9|40|800|27|4800|1.2|1.4|14|40|
| B 16-17|3020|61.5|50|800|28|4800|1.5|1.8|17|40|
| G 16-17|2440|55.5|35|800|26|4800|1.|1.2|14|40|
| M|2730|60|30|600|17|4800|1.4|1.4|18|40|
| W|2230|55|25|1200|21|4800|1.1|1.3|14|40|



### Nutritional Needs of Households



Our data on demand and nutrients is at the *household* level; we
   can&rsquo;t directly compare household level nutrition with individual
   level requirements.  What we **can** do is add up minimum individual
   requirements, and see whether household total exceed these.  This
   isn&rsquo;t a guarantee that all individuals have adequate nutrition
   (since the way food is allocated in the household might be quite
   unequal, or unrelated to individual requirements), but it is
   *necessary* if all individuals are to have adequate nutrition.

For the average household in the ICRISAT villages, the number of
different kinds of people can be computed by averaging over households:



In [1]:
# In first round, averaged over households and villages
zbar = r.z.sel(t=r.firstround,drop=True).mean(['j','m'])[:-1] # Leave out log HSize

Now, the inner/dot/matrix product between `zbar` and the `rda`
DataFrame of requirements will give us minimum requirements for the
average household:



In [1]:
rdi = eep153_tools.read_sheets(key,
                               json_creds='../students.json',
                               sheet='RDI').set_index('k').replace(np.nan,0)

# This matrix product gives minimum nutrient requirements for average
# household in 1975.
hh_rdi = rdi.replace('',0).T@zbar

# But this is per *day*, while our ICRISAT food data is per *year*:
hh_rdi = hh_rdi*365.25

## Nutritional Adequacy of Food Demands



Since we can trace out demands for nutrients as a function of $(x,p)$,
and we&rsquo;ve computed minimum nutritional requirements for the average
household, we can *normalize* nutritional intake to check the adequacy
of diet.



In [1]:
def nutrient_adequacy_ratio(x,p):
    return nutrient_demand(x,p)/hh_rdi

In terms of normalized nutrients, any household with more than one
unit of any given nutrient (or zero in logs) will be consuming a
minimally adequate level of the nutrient; below this level there&rsquo;s
clearly nutritional inadequacy.  For this reason the ratio of
actual nutrients to required nutrients is termed the &ldquo;nutrient
adequacy ratio,&rdquo; or NAR.



In [1]:
plt.plot(X,pd.concat({x:np.log(nutrient_adequacy_ratio(x,p))[UseNutrients] for x in X},axis=1).T)
plt.legend(UseNutrients)
plt.xlabel('log budget')
plt.ylabel('log nutrient adequacy ratio')
plt.axhline(0)

As before, we can also vary relative prices.  Here we trace out
nutritional adequacy varying the price of a single good:



In [1]:
poorer_x = median_x/2

plt.plot(pd.concat({p0:np.log(nutrient_adequacy_ratio(poorer_x,my_prices(p0,i=USE_GOOD)))[UseNutrients] for p0 in P},axis=1).T,P)
plt.legend(UseNutrients)
plt.ylabel('Price')
plt.xlabel('log nutrient adequacy ratio')
plt.axvline(0)
plt.axhline(p.sel(i=USE_GOOD).values)